# Modelo definitivo

CHONA ME DIJO QUE USARA LOS DATOS SIN PCA Y QUE HAGA UN POOLING GRANDE AL PRINCIPIO DE LA CNN PARA ACHICAR LOS DATOS DE UNA CUANDO ENTRAN. 
ADEMÁS, TENGO QUE HACER CONV2D EN VES DE 1D Y TENGO QUE AGREGARLE LOS NUEVOS DATOS. ADEMÁS TENGO QUE USAR EL PRETRAINED MODEL RESNET QUE USO 
EL INDIO EN EL VIDEO PARA TENER YO YA HECHO EL COSO Y CON LOS WEIGHTS Y QUE NO ME TRADE TANTO EL ENTRENAMIENTO. YO LE CAMBIARÍA LO DEL POOL AL 
PRINCIPIO Y LO ADAPTARÍA AL FINAL DE LA RED PARA QUE SE AJUSTE A MIS DATOS.
- TENGO QUE CORRER EL CODIGO PARA EL DATASET DE NUEVO PERO SIN EL PCA 
- VOLVER A ARREGLAR LA CNN PARA QUE ADMITA LA VIEJA INPUT SHAPE
- USAR UN MODELO RESNET PREENTRENADO CON DATOS COHERENTES (PREFERENTEMENTE EL QUE USÓ EL INDIO)
- INCORPORAR LO QUE ME DIJO CHONA DE CONV2D Y UN POOLING PARA QUE SE ADAPTE A RESNET (AdaptiveAvgPool2d - https://stackoverflow.com/questions/61955991/change-input-shape-dimensions-for-resnet-model-pytorch)

CODIGOS DEL INDIO:
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg-conv2d.ipynb
https://github.com/talhaanwarch/youtube-tutorials/blob/main/eeg_epilepsy.ipynb

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import tensorflowjs
import pandas as pd
import numpy as np

In [ ]:
X = np.load('C:\\Users\\47575909\\Desktop\\X.npy')
y = np.load('C:\\Users\\47575909\\Desktop\\y.npy')

In [ ]:
#DIVIDO EN TRAIN, TEST Y EVALUATION
#Divido los datos de train de los de test y evaluation (70/30)
X_train, X_testVal, y_train, y_testVal = train_test_split(X, y, test_size=0.3, random_state=42)

#Divido los datos de test de los de evaluation (50/50 del 30% anterior)
X_val, X_test, y_val, y_test = train_test_split(X_testVal, y_testVal, test_size=0.5, random_state=42)

In [ ]:
#Forma de mis datos
input_tensor = Input(shape=(14, 230, 384))

#Cargo el modelo ResNet50 preentrenado
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

#Congelo las capas del modelo base para que no se actualicen en el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

#Agrego GlobalAveragePooling2D para adaptar la salida del modelo base
x = GlobalAveragePooling2D()(base_model.output)

#Agrego capas adicionales para la clasificación de mis datos
x = Conv2D(filters=32, kernel_size=3, activation='relu')
x = MaxPooling2D(pool_size=2)
x = Conv2D(filters=32, kernel_size=3, activation='relu')
x = MaxPooling2D(pool_size=2)
x = Flatten()'''(base_model.output)'''
x = Dense(16, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(8, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

#Junto todo y creo el modelo
model = Model(inputs=input_tensor, outputs=output)



Conv2D(filters=32, kernel_size=3, activation='relu'),
MaxPooling2D(pool_size=2),

In [ ]:
#COMPILO
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#ENTRENO
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

#EVALÚO
test_loss, test_acc = model.evaluate(X_val, y_val)

In [ ]:
#VEO LA PERFORMANCE
print("Accuracy : ", test_acc)
print("Loss : ", test_loss)

In [ ]:
#HAGO UNA PREDICCIÓN
prediction = model.predict(datos_test)
epilepsy_prob = predicciones[:, 1]
print("Probabilidad de tener epilepsia:", epilepsy_prob)

##### Decirle a chona que el indio usa ResNet-26D y que keras no lo tiene ese. Que hago?...
- Uso otro modelo de ResNet
- Uso las liberías pytorch y timm que si tienen el ResNet-26D

# Exportamos el modelo
(Con tensorflow.js)

In [ ]:
#Exportamos el modelooo
tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve /ruta/al/modelo/saved_model /ruta/de/destino/
#che, no entiendo lo de las rutas

In [ ]:
//HABRÍA QUE HACER ESTO PARA QUE ME DEVUELVA EN LA WEB SOLO LA PROBABILIDAD QUE YO QUIERO (la probabilidad de tener epilepsia)

/*// Obtener datos de entrada (pueden ser simulados en este ejemplo)
const datosEntrada = obtenerDatosDeEntrada();

// Cargar el modelo
const modelo = await cargarModelo();

// Realizar predicciones
const predicciones = modelo.predict(tf.tensor(datosEntrada));

// Mostrar la probabilidad de tener epilepsia
const probabilidadEpilepsia = predicciones.dataSync()[1];
console.log('Probabilidad de tener epilepsia:', probabilidadEpilepsia);*/